In [ ]:
!pip install mysql-connector-python==8.0.29

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 51.1 MB/s eta 0:00:00


In [ ]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
import statsmodels.api as sm

import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import mysql.connector

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
cnx = mysql.connector.connect(host = 'analytics-cluster.cluster-ro-c3e1i1vd2jfw.us-east-1.rds.amazonaws.com',
                              port = '3306',
                              user = 'analytics',
                              password = 'Rb4v9CTO75ywQlcwdDsW8PHEupvS7T4bN4tXvKgg',
                              database = '8base_5e5c332533f7f90008001e0b'
                                     )

# Query

In [ ]:
df = pd.read_sql_query(''' SELECT i.isCompleted,i.latestInteraction,
t.id, t.cashAmount, t.currency,iit.minMarketPrice, ii.tradeWillingness,ii.wasPreviouslyAuthenticated, sq.size, sq.hasBox, sq.boxStatus,sq.hasAccessories,
t1.id,t1.cashAmount,t1.currency,iit1.minMarketPrice,ii1.tradeWillingness,ii1.wasPreviouslyAuthenticated,sq1.size,sq1.hasBox,sq1.boxStatus,sq1.hasAccessories,
CASE WHEN wm.id IS NOT NULL THEN 1 ELSE 0 END AS wishlistMatch
FROM WishlistMatch AS wm
INNER JOIN UsersWishlistMatches1 AS uwm
ON wm.id = uwm.wishlistMatchBId

INNER JOIN Users AS u
ON uwm.userAId = u.id

INNER JOIN InteractionUsers AS iu
ON u.id = iu.userBId
INNER JOIN InteractionsUsers1 AS iu1
ON u.id = iu1.userBId

RIGHT JOIN Interactions as i
ON iu.interactionAId = i.id OR iu1.interactionAId = i.id

INNER JOIN InteractionTerm AS it
ON i.id = it.interactionBId
INNER JOIN InteractionTerm1 as it1
ON i.id = it1.interactionBId

INNER JOIN Terms as t
ON t.id = it.termAId
INNER JOIN Terms as t1
ON t1.id = it1.termAId

INNER JOIN InventoryItemTermTerms AS iitt
ON iitt.termBId = t.id
INNER JOIN InventoryItemTermTerms AS iitt1
ON iitt1.termBId = t1.id

INNER JOIN InventoryItemTerms AS iit
ON iitt.inventoryItemTermAId = iit.id
INNER JOIN InventoryItemTerms AS iit1
ON iitt1.inventoryItemTermAId = iit1.id

INNER JOIN InventoryItemTermInventoryItems AS iitii
ON iitii.inventoryItemTermAId = iit.id
INNER JOIN InventoryItemTermInventoryItems AS iitii1
ON iitii1.inventoryItemTermAId = iit1.id

INNER JOIN InventoryItems AS ii
ON iitii.inventoryItemBId = ii.id
INNER JOIN InventoryItems AS ii1
ON iitii1.inventoryItemBId = ii1.id

INNER JOIN InventoryItemInventoryQuality AS iiiq
ON ii.id = iiiq.inventoryItemAId
INNER JOIN InventoryItemInventoryQuality AS iiiq1
ON ii1.id = iiiq1.inventoryItemAId

INNER JOIN InventoryQualities AS iq
ON iiiq.inventoryQualityBId = iq.id
INNER JOIN InventoryQualities AS iq1
ON iiiq1.inventoryQualityBId = iq1.id

INNER JOIN InventoryQualityShoeQuality as iqsq
ON iq.id = iqsq.inventoryQualityAId
INNER JOIN InventoryQualityShoeQuality as iqsq1
ON iq1.id = iqsq1.inventoryQualityAId

INNER JOIN ShoeQualities as sq
ON iqsq.shoeQualityBId = sq.id
INNER JOIN ShoeQualities as sq1
ON iqsq1.shoeQualityBId = sq1.id

LIMIT 5
''', cnx)


/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
df = pd.read_sql_query(''' SELECT i.isCompleted,i.latestInteraction,
t.id, t.cashAmount, t.currency,iit.minMarketPrice, ii.tradeWillingness,ii.wasPreviouslyAuthenticated, sq.size, sq.hasBox, sq.boxStatus,sq.hasAccessories,
t1.id,t1.cashAmount,t1.currency,iit1.minMarketPrice,ii1.tradeWillingness,ii1.wasPreviouslyAuthenticated,sq1.size,sq1.hasBox,sq1.boxStatus,sq1.hasAccessories,
i.id AS interactionId , gg.id AS givingShoe, gg1.id AS receivingShoe

FROM Interactions as i

INNER JOIN InteractionTerm AS it
ON i.id = it.interactionBId

INNER JOIN InteractionTerm1 as it1
ON i.id = it1.interactionBId

INNER JOIN Terms as t
ON t.id = it.termAId
INNER JOIN Terms as t1
ON t1.id = it1.termAId

INNER JOIN InventoryItemTermTerms AS iitt
ON iitt.termBId = t.id
INNER JOIN InventoryItemTermTerms AS iitt1
ON iitt1.termBId = t1.id

INNER JOIN InventoryItemTerms AS iit
ON iitt.inventoryItemTermAId = iit.id
INNER JOIN InventoryItemTerms AS iit1
ON iitt1.inventoryItemTermAId = iit1.id

INNER JOIN InventoryItemTermInventoryItems AS iitii
ON iitii.inventoryItemTermAId = iit.id

INNER JOIN InventoryItemTermInventoryItems AS iitii1
ON iitii1.inventoryItemTermAId = iit1.id

INNER JOIN InventoryItems AS ii
ON iitii.inventoryItemBId = ii.id
INNER JOIN InventoryItems AS ii1
ON iitii1.inventoryItemBId = ii1.id

INNER JOIN GenericGoodInventoryItems AS ggii
ON ggii.inventoryItemAId = ii.id
INNER JOIN GenericGoodInventoryItems AS ggii1
ON ggii1.inventoryItemAId = ii1.id

INNER JOIN GenericGoods AS gg
ON ggii.genericGoodBId = gg.id
INNER JOIN GenericGoods AS gg1
ON ggii1.genericGoodBId = gg1.id


INNER JOIN InventoryItemInventoryQuality AS iiiq
ON ii.id = iiiq.inventoryItemAId
INNER JOIN InventoryItemInventoryQuality AS iiiq1
ON ii1.id = iiiq1.inventoryItemAId

INNER JOIN InventoryQualities AS iq
ON iiiq.inventoryQualityBId = iq.id
INNER JOIN InventoryQualities AS iq1
ON iiiq1.inventoryQualityBId = iq1.id

INNER JOIN InventoryQualityShoeQuality as iqsq
ON iq.id = iqsq.inventoryQualityAId
INNER JOIN InventoryQualityShoeQuality as iqsq1
ON iq1.id = iqsq1.inventoryQualityAId

INNER JOIN ShoeQualities as sq
ON iqsq.shoeQualityBId = sq.id
INNER JOIN ShoeQualities as sq1
ON iqsq1.shoeQualityBId = sq1.id

''', cnx)

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Capstone/df_model_final.csv'
df = pd.read_csv(path)

In [ ]:
df.describe()

,Unnamed: 0,isCompleted,cashAmount_x,minMarketPrice_x,tradeWillingness_x,wasPreviouslyAuthenticated_x,size_x,hasBox_x,hasAccessories_x,cashAmount_y,minMarketPrice_y,tradeWillingness_y,wasPreviouslyAuthenticated_y,size_y,hasBox_y,hasAccessories_y,Sum_Wishlist_Match
count,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06,6.544458e+06
mean,3.272228e+06,1.713923e-02,1.255347e+01,6.340469e+02,1.301846e+00,3.245982e-01,1.043681e+01,1.301775e-01,9.573792e-01,1.255347e+01,6.340469e+02,1.301846e+00,3.245982e-01,1.043681e+01,1.301775e-01,9.573792e-01,1.295110e-01
std,1.889222e+06,1.297902e-01,1.069964e+02,9.616775e+04,6.291486e-01,4.682245e-01,1.633879e+00,3.364986e-01,2.020007e-01,1.069964e+02,9.616775e+04,6.291486e-01,4.682245e-01,1.633879e+00,3.364986e-01,2.020007e-01,1.294079e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,3.700000e+01,0.000000e+00,0.000000e+00,3.500000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.700000e+01,0.000000e+00,0.000000e+00,3.500000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.636114e+06,0.000000e+00,0.000000e+00,2.339700e+02,1.000000e+00,0.000000e+00,9.500000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.339700e+02,1.000000e+00,0.000000e+00,9.500000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,3.272228e+06,0.000000e+00,0.000000e+00,3.300000e+02,1.000000e+00,0.000000e+00,1.050000e+01,0.000000e+00,1.000000e+00,0.000000e+00,3.300000e+02,1.000000e+00,0.000000e+00,1.050000e+01,0.000000e+00,1.000000e+00,0.000000e+00
75%,4.908343e+06,0.000000e+00,0.000000e+00,4.995200e+02,2.000000e+00,1.000000e+00,1.150000e+01,0.000000e+00,1.000000e+00,0.000000e+00,4.995200e+02,2.000000e+00,1.000000e+00,1.150000e+01,0.000000e+00,1.000000e+00,0.000000e+00
max,6.544457e+06,1.000000e+00,5.000000e+03,1.000000e+08,2.000000e+00,1.000000e+00,1.800000e+01,1.000000e+00,1.000000e+00,5.000000e+03,1.000000e+08,2.000000e+00,1.000000e+00,1.800000e+01,1.000000e+00,1.000000e+00,4.050000e+02


In [ ]:
df = df.drop('Unnamed: 0',axis = 1)

# Differentiating Column Names between User_x and User_y

User_x is the user proposing a trade, User_y is the user receiving the proposal (what user 1 would recieve from the proposed trade.

In [ ]:
new_names = ['isCompleted','latestInteraction',
             'id_x','cashAmount_x','currency_x','minMarketPrice_x','tradeWillingness_x','wasPreviouslyAuthenticated_x','size_x','hasBox_x','boxStatus_x','hasAccessories_x',
             'id_y','cashAmount_y','currency_y','minMarketPrice_y','tradeWillingness_y','wasPreviouslyAuthenticated_y','size_y','hasBox_y','boxStatus_y','hasAccessories_y',
             'interactionId','givingShoe_x','receivingShoe_y','wishlist_matches']


In [ ]:
def rename_columns(df, new_names):
    column_names = df.columns.values
    for i in range(len(column_names)):
        column_names[i] = new_names[i]
    df.columns = column_names

rename_columns(df,new_names)

In [ ]:
df.shape

(6544458, 26)

# Filtering and Cleaning for Unique Interactions

In [ ]:
def filter_rows(df):
    """
    Filter out rows with 'None' in both columns
    Creates a unique id for same interaction between users
    """
    # Filter out rows with None in both columns
    df_filtered = df[df.minMarketPrice_x.notnull() & df.minMarketPrice_y.notnull()]
    
    # Create unique id for same interaction between users
    df_filtered['temp_id'] = df_filtered[['id_x', 'id_y']].apply(lambda x: ''.join(sorted(x)), axis=1)
    df_filtered = df_filtered[df_filtered.temp_id == df_filtered.id_x + df_filtered.id_y]
    
    return df_filtered

In [ ]:
temp = filter_rows(df)

NameError: ignored

In [ ]:
temp = temp.reset_index()

In [ ]:
temp = temp.drop('index',axis=1)

In [ ]:
#from google.colab import files
#df_filt.to_feather('df_trades.feather')
#files.download('df_trades.feather')

# Total Trade Value, Total Shoe Value for x and y users

In [ ]:
#convert minMarketPrice_x, cashAmount_x, minMarketPrice_y, and cashAmount_y columns to float
def convert_float(df):
  df['minMarketPrice_x'] = df['minMarketPrice_x'].astype(float)
  df['minMarketPrice_y'] = df['minMarketPrice_y'].astype(float)
  df['cashAmount_x'] = df['cashAmount_x'].astype(float)
  df['cashAmount_y'] = df['cashAmount_y'].astype(float)
  return df

In [ ]:
def calculate_total_shoe_values(df):
  df['tot_shoe_value_x'] = df.groupby('id_x')['minMarketPrice_x'].transform(lambda x: x.sum() if x.nunique() > 1 else x.iloc[0])
  df['tot_shoe_value_y'] = df.groupby('id_y')['minMarketPrice_y'].transform(lambda x: x.sum() if x.nunique() > 1 else x.iloc[0])
  return df

In [ ]:
def calculate_total_trade_values(df):
  df['tot_trade_val_x'] = df['tot_shoe_value_x'] + df['cashAmount_x']
  df['tot_trade_val_y'] = df['tot_shoe_value_y'] + df['cashAmount_y']
  return df

In [ ]:
def calculate_trade_value_difference(df):
  df['trade_val_diff'] = df['tot_trade_val_x'] - df['tot_trade_val_y']
  return df

In [ ]:
def create_target_variable(df):
  df['latestInteraction'] = df['latestInteraction'].astype(str)
  df_new = df[~df['latestInteraction'].isin(['userReviewPending', 'confirmed', 'cancelled', 'sent'])]
  df_new['latestInteraction'] = df_new['latestInteraction'].replace({'rejected': 0, 'countered': 0, 'accepted': 1})
  return df_new

In [ ]:
def transform_data(df):
  df = convert_float(df)
  df = calculate_total_shoe_values(df)
  df = calculate_total_trade_values(df)
  df = calculate_trade_value_difference(df)
  df = create_target_variable(df)
  return df

In [ ]:
df_1 = transform_data(temp)

KeyError: ignored

In [ ]:
df_1[(df_1['latestInteraction']==1)&(df_1['wishlist_matches']>0)].shape

In [ ]:
df_1[(df_1['latestInteraction']==0)&(df_1['wishlist_matches']==0)].shape

# Creating Dataset for Model

In [ ]:
df_1.shape

NameError: ignored

## Filter out bad offers

In [ ]:
def filter_bad_offers(df):
    #acc_df_1 = df[df['latestInteraction']==1]
    #avg_acc_diff = acc_df_1['trade_val_diff'].mean()
    #df_mask = df[(df['latestInteraction']== 0) & (df['trade_val_diff'] <= 3*avg_acc_diff)]
    #df_mask_1 = df[df['latestInteraction']==1]
    df_new = df[(df['trade_val_diff']>-2000) & (df['trade_val_diff'] < 2000)]
    return df_new

In [ ]:
df_2 = filter_bad_offers(df_1)

In [ ]:
df_2.shape

In [ ]:
df_2['size_match'] = np.where(df_2['size_x'] == df_2['size_y'], 1, 0)

## Model-ready Dataset

In [ ]:
def model_dataset(df):
  df_new = pd.DataFrame()
  grouped_df3 = df.groupby(['temp_id'])

  df_new['isCompleted'] = grouped_df3['isCompleted'].unique().astype(int)
  df_new['latestInteraction'] = grouped_df3['latestInteraction'].unique().astype(int)
  df_new['trade_val_diff'] = grouped_df3['trade_val_diff'].unique().astype(float)
  df_new['tradeWillingness_x'] = grouped_df3['tradeWillingness_x'].mean()
  df_new['hasBox_x'] = grouped_df3['hasBox_x'].mean()
  df_new['hasAccessories_x'] = grouped_df3['hasAccessories_x'].mean()
  df_new['tradeWillingness_y'] = grouped_df3['tradeWillingness_y'].mean()
  df_new['hasBox_y'] = grouped_df3['hasBox_y'].mean()
  df_new['hasAccessories_y'] = grouped_df3['hasAccessories_y'].mean()
  df_new['size_match'] = grouped_df3['size_match'].mean()
  df_new['wishlist_matches'] = grouped_df3['wishlist_matches'].unique().astype(float)

  return df_new

In [ ]:
df3 = model_dataset(df_2)

In [ ]:
df3 = df3.reset_index()

In [ ]:
df3['wishlist_matches'].value_counts().sort_values()

In [ ]:
df3 = df3[df3['wishlist_matches'] <= 5]

In [ ]:
df3['wishlist_matches'].value_counts().sort_values()

In [ ]:
df3 = df3.reset_index()

In [ ]:
df3 = df3.drop('index',axis=1)

In [ ]:
#Compute the correlation matrix
corr = df3.corr()

# Visualize the correlation matrix using seaborn
sns.heatmap(corr,
              xticklabels=corr.columns.values,
              yticklabels=corr.columns.values)

# Download Data

In [ ]:
from google.colab import files
#df3.to_csv('df3.csv', encoding = 'utf-8-sig') 
#files.download('df3.csv')
df3.to_feather('model_ready_df.feather')
files.download('model_ready_df.feather')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>